# AzionRetriever

## 概述

这将帮助你快速开始使用 [AzionRetriever](/docs/concepts/#retrievers)。如需关于 AzionRetriever 所有功能和配置的详细文档，请前往 [API 参考文档](https://api.js.langchain.com/classes/_langchain_community.retrievers_azion_edgesql.AzionRetriever.html)。

### 集成详情


| 检索器 | 自托管 | 云服务 | 包 | [Python 支持] |
| :--- | :---: | :---: | :---: | :---: |
[AzionRetriever](https://api.js.langchain.com/classes/_langchain_community.retrievers_azion_edgesql.AzionRetriever.html) | ❌ | ❌ | @langchain/community | ❌ |


## 准备

要使用 AzionRetriever，你需要设置 AZION_TOKEN 环境变量。

```typescript
process.env.AZION_TOKEN = "your-api-key"
```

如果你在本指南中使用 OpenAI 嵌入，请同时设置你的 OpenAI 密钥：

```typescript
process.env.OPENAI_API_KEY = "YOUR_API_KEY";
```
如果你希望从单个查询中获取自动跟踪信息，也可以取消以下代码行的注释并设置你的 [LangSmith](https://docs.smith.langchain.com/) API 密钥：

```typescript
// process.env.LANGSMITH_API_KEY = "<YOUR API KEY HERE>";
// process.env.LANGSMITH_TRACING = "true";
```

### 安装

该检索器位于 `@langchain/community/retrievers/azion_edgesql` 包中：

```{=mdx}
import IntegrationInstallTooltip from "@mdx_components/integration_install_tooltip.mdx";
import Npm2Yarn from "@theme/Npm2Yarn";

<IntegrationInstallTooltip></IntegrationInstallTooltip>

<Npm2Yarn>
  azion @langchain/openai @langchain/community
</Npm2Yarn>
```

## 实例化

现在我们可以实例化检索器：

In [56]:
import { AzionRetriever } from "@langchain/community/retrievers/azion_edgesql";
import { OpenAIEmbeddings } from "@langchain/openai";
import { ChatOpenAI } from "@langchain/openai";

const embeddingModel = new OpenAIEmbeddings({
  model: "text-embedding-3-small"
})

const chatModel = new ChatOpenAI({
  model: "gpt-4o-mini",
  apiKey: process.env.OPENAI_API_KEY
})

const retriever = new AzionRetriever(embeddingModel, 
  {dbName:"langchain",
   vectorTable:"documents", // table where the vector embeddings are stored
   ftsTable:"documents_fts", // table where the fts index is stored
   searchType:"hybrid", // search type to use for the retriever
   ftsK:2, // number of results to return from the fts index
   similarityK:2, // number of results to return from the vector index
   metadataItems:["language","topic"],
   filters: [{ operator: "=", column: "language", value: "en" }],
   entityExtractor:chatModel

}) // number of results to return from the vector index

## 用法

In [57]:
const query = "Australia"

await retriever.invoke(query);

[
  Document {
    pageContent: 'Australia s indigenous people have inhabited the continent for over 65,000 years',
    metadata: { language: 'en', topic: 'history', searchtype: 'similarity' },
    id: '3'
  },
  Document {
    pageContent: 'Australia is a leader in solar energy adoption and renewable technology',
    metadata: { language: 'en', topic: 'technology', searchtype: 'similarity' },
    id: '5'
  },
  Document {
    pageContent: 'Australia s tech sector is rapidly growing with innovation hubs in major cities',
    metadata: { language: 'en', topic: 'technology', searchtype: 'fts' },
    id: '7'
  }
]


## 在链式应用中使用

与其他检索器一样，AzionRetriever 可以通过 [链式应用](/docs/how_to/sequence/) 集成到 LLM 应用程序中。

我们需要一个 LLM 或聊天模型：

```{=mdx}
import ChatModelTabs from "@theme/ChatModelTabs";

<ChatModelTabs customVarName="llm" />
```

In [43]:
// @lc-docs-hide-cell

import { ChatOpenAI } from "@langchain/openai";

const llm = new ChatOpenAI({
  model: "gpt-4o-mini",
  temperature: 0,
});

In [44]:
import { ChatPromptTemplate } from "@langchain/core/prompts";
import { RunnablePassthrough, RunnableSequence } from "@langchain/core/runnables";
import { StringOutputParser } from "@langchain/core/output_parsers";

import type { Document } from "@langchain/core/documents";

const prompt = ChatPromptTemplate.fromTemplate(`
Answer the question based only on the context provided.

Context: {context}

Question: {question}`);

const formatDocs = (docs: Document[]) => {
  return docs.map((doc) => doc.pageContent).join("\n\n");
}

// See https://js.langchain.com/docs/tutorials/rag
const ragChain = RunnableSequence.from([
  {
    context: retriever.pipe(formatDocs),
    question: new RunnablePassthrough(),
  },
  prompt,
  llm,
  new StringOutputParser(),
]);

In [45]:
await ragChain.invoke("Paris")

The context mentions that the 2024 Olympics are in Paris.


## API 参考文档

如需详细了解 AzionRetriever 的所有功能和配置，请访问 [API 参考文档](https://api.js.langchain.com/classes/_langchain_community.retrievers_azion_edgesql.AzionRetriever.html)。
